In [ ]:
import json
import os
import shutil
%cd yolov5

In [ ]:
"""
folder를 만드는 과정
"""

def make_directory(pth):
    if not os.listdir(pth):
        ann_pth_tr = pth + "\\" + "train"
        ann_pth_val = pth + "\\" + "valid"
        os.mkdir(ann_pth_tr)
        os.mkdir(ann_pth_val)
    return
make_directory("./data/labels")
make_directory("./data/images")

In [ ]:
with open("./data/train.json", "r", encoding = "utf-8") as f:
    data = json.load(f)
data[0].keys()

- image key format
    - {'date': '20201012',
     'path': 'S2-N1104M00001',
     'filename': 'S2-N1104M01957.jpg',
     'copyrighter': '미디어그룹사람과숲(컨)',
     'H_DPI': 96,
     'location': '11',
     'V_DPI': 96,
     'bit': '24',
     'resolution': [1920, 1080]}
- annotation format
    - [{'data ID': 'S2',
      'middle classification': '01',
      'flags': 'not occluded, not truncated',
      'box': [776, 517, 839, 578],
      'class': '02'},
     {'data ID': 'S2',
      'middle classification': '01',
      'flags': 'not occluded, not truncated',
      'box': [773, 598, 845, 645],
      'class': '06'},
     {'data ID': 'S2',
      'middle classification': '01',
      'flags': 'not occluded, not truncated',
      'box': [1014, 354, 1072, 423],
      'class': '07'},
     {'data ID': 'S2',
      'middle classification': '01',
      'flags': 'occluded, not truncated',
      'box': [974, 416, 1111, 583],
      'class': '01'},
     {'polygon': [[1081, 519],
       [1074, 520],
       [1071, 525],
       [1072, 531],
       [1076, 536],
       [1082, 537]],
      'data ID': 'S2',
      'middle classification': '01',
      'flags': 'not occluded, not truncated',
      'class': '04'},
     {'data ID': 'S2',
      'middle classification': '01',
      'flags': 'not occluded, not truncated',
      'box': [1025, 667, 1113, 762],
      'class': '05'},
     {'data ID': 'S2',
      'middle classification': '07',
      'flags': 'not occluded, not truncated',
      'box': [625, 554, 690, 677],
      'class': '41'}]


- 데이터 정의서
    - image: 1. path: path 정보 2. filename: 파일 이름 3. resolution: 이미지 해상도
    - annotations: 대분류: 대분류 정보 / 중분류: 중분류 정보 / flag: 겹침 및 잘림 여부 / box: Bounding Box 좌표 / class: class 정보
- image에서 활용: filename, annotation에서 활용: box, class

In [ ]:
"""
Annotation 정보 중 belt의 경우 polygon 형태로 저장된 상태이다.
polygon으로 저장된 정보를 bounding box의 형태로 저장한다.
"""

def polygon_to_bbx(p, w, h):
    flag = True
    if type(p[0]) == int:
        flag = False
        return flag
    if flag:
        x_ = [point[0] for point in p]
        y_ = [point[1] for point in p]
        xmin = min(x_)
        xmax = max(x_)
        ymin = min(y_)
        ymax = max(y_)

        x_c = (xmin + xmax) / (2 * w)
        y_c = (ymin + ymax) / (2 * h)
        b_w = (xmax - xmin) / w
        b_h = (ymax - ymin) / h
        return x_c, y_c, b_w, b_h

In [ ]:
"""
class 정보, bounding box 정보를 담아야 한다
polygon: 다각형 정보
Bounding Box 좌표 형태: xmin, ymin, xmax, ymax 형태으로 저장된 것을 X_center, Y_center, Width, Height 형태로 변경하여 저장.
image의 크기를 바탕으로 정규화 하여 저장한다.
class 정보 0, 1, 2, 3, 4, 5, 6으로 지정. 순서대로 0,1은 각각 안전벨트 착용, 미착용 / 2, 3은 안전화 착용, 미착용 / 4, 5는 안전모 착용 미착용.
"""
class_info = [1, 2, 5, 6, 7, 8]
def make_labels(d):
    image_info = d["image"]
    annot_info = d["annotations"]
    filename = d["image"]["filename"][:-4]
    width = image_info["resolution"][0]
    height = image_info["resolution"][1]
    label_file_pth = "./dataset/labels"
    label_file_path = os.path.join(label_file_pth, filename + ".txt")
    with open(label_file_path, "w") as f:
        for annot in annot_info:
            class_no = int(annot["class"])
            if class_no not in class_info:
                continue
            # class를 0부터 6까지 맞추기 위함.
            # ai hub의 데이터셋은 안전 장구류 외의 구조물 등의 경우도 존재. 안전 장구류 클래스만 포함시키기 위해서 다음과 같은 작업을 진행
            if class_no < 3:
                class_id = str(class_no - 1)
            else:
                class_id = str(class_no - 3)
            annot_list = list(annot.keys())
            if "box" in annot_list:
                bbx = annot["box"]
                xmin, ymin, xmax, ymax = bbx[0], bbx[1], bbx[2], bbx[3]
                # normalization 과정을 거친다.
                x_center = (xmin+xmax) / (2*width)
                y_center = (ymin+ymax) / (2*height)
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
            if "polygon" in annot_list:
                if not annot["polygon"]:
                    raise ValueError("Empty Polygon")
                polygon = polygon_to_bbx(annot["polygon"], width, height)
                if not polygon:
                    continue
                x_center, y_center, w, h = polygon[0], polygon[1], polygon[2], polygon[3]
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
        print("Make label file {}".format(label_file_path))
for d in data:
    make_labels(d)

### 추려낸 이미지에 대해서 label을 찾는 부분이다.
- image를 train, valid 폴더에 각각 분리해서 저장
- image 폴더에서 각각 train 부분, valid 부분에서 파일명을 따온다
- label 정보의 이름과 image의 이름이 같도록 구성한다

In [ ]:
"""
make_label 함수는 label을 만드는 과정이다.
밑의 과정들을 한번에 함수로 묶은 것이다.
"""
def make_label(mode):
    image_root = os.path.join("./dataset/images", mode)
    images = os.listdir(image_root)
    image_name = []
    for i in images:
        image_name.append(i[:-4])
    label_root = "./dataset/labels"
    label = os.listdir(label_root)
    in_image = []
    for i in label:
        if i[:-4] in image_name:
            in_image.append(i)
    label_root = "./dataset/labels"
    label = os.listdir(label_root)
    in_image = []
    for i in label:
        if i[:-4] in image_name:
            in_image.append(i)
    """
    생성한 label 디렉터리의 train folder에 넣어준다.
    """
    for text in in_image:
        from_pth = os.path.join(label_root, text)
        to_pth = "./dataset/labels/train"
        shutil.copy(from_pth, to_pth)

make_label("train")
make_label("valid")

In [ ]:
image_root = "./dataset/images/train/"
images = os.listdir(image_root)
image_name = []
for i in images:
    image_name.append(i[:-4])
image_name

In [ ]:
label_root = "./dataset/labels"
label = os.listdir(label_root)
in_image = []
for i in label:
    if i[:-4] in image_name:
        in_image.append(i)
in_image

In [ ]:
"""
생성한 label 디렉터리의 train folder에 넣어준다.
"""
for text in in_image:
    from_pth = os.path.join(label_root, text)
    to_pth = "./dataset/labels/train"
    shutil.copy(from_pth, to_pth)

In [ ]:
image_root = "./dataset/images/valid/"
images = os.listdir(image_root)
image_name = []
for i in images:
    image_name.append(i[:-4])
image_name

In [ ]:
cnt = 0
label_root = "./dataset/labels"
label = os.listdir(label_root)
in_image = []
out_image = []
for i in label:
    if i[:-4] in image_name:
        in_image.append(i)
    if i[:-4] not in image_name:
        out_image.append(i)

In [ ]:
for text in in_image:
    from_pth = os.path.join(label_root, text)
    to_pth = "./dataset/labels/valid"
    shutil.copy(from_pth, to_pth)

In [ ]:
name_imgs = []
for imgs in in_image:
    name_imgs.append(imgs[:-4] + ".jpg")
name_imgs

In [ ]:
"""
yaml file의 구성
train: train image file의 저장 경로
val: valid image file의 저장 경로
nc: number of class -> train 시에 class의 수를 지정
names: class의 이름
cfg.yaml이라는 이름으로 저장
train 시에 cfg.yaml을 이용한다.
"""

import yaml
data = {'train': "./dataset/images/train",
        "val": "./dataset/images/valid",
        "nc": 6,
        "names": ['Hard', 'No Hard', "Helmet", "No Helmet", "Belt", "No Belt"]}

with open("cfg.yaml", "w") as f:
    yaml.dump(data, f)

- 여기까지가 label을 구성하는 단계이다.
- 아래부터는 + alpha 느낌으로 데이터를 추가할 시에 label의 class 정보를 cfg.yaml에 맞게 저장하는 과정이다.

In [ ]:
pth = "./dataset/labels/valid"
pth_tr = "./dataset/images/train"
mv_pth = "./dataset/images/valid"
label_val = os.listdir(pth)
for name in label_val:
    name = name[:-4]
    root = os.path.join(pth_tr, name+".jpg")
    shutil.move(root, mv_pth)

In [ ]:
import os
new_annot_train = "./dataset/labels/train"
new_annot_valid = "./dataset/labels/valid"
new_annot_tr = os.listdir(new_annot_train)
new_annot_val = os.listdir(new_annot_valid)
for annot in new_annot_tr:
    if annot[:5] == "PartB":
        ann_pth = os.path.join(new_annot_train, annot)
        with open(ann_pth, "r") as f:
            data = f.readlines()
        with open(ann_pth, "w") as f:
            for i in data:
                ann = i.split()
                ann[0] = int(ann[0]) + 2
                class_id, x_center, y_center, w, h = ann[0], float(ann[1]), float(ann[2]), float(ann[3]), float(ann[4])
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")